In [ ]:
# 🛠️ Install required libraries
!pip install requests pandas

# 📦 Imports
import requests
import time
import pandas as pd
import os

# 🔑 Your SerpAPI Key
API_KEY = "72091d6981c17f40f4a93d1cab3081f67e66f2424f6237a7f9f4b3463933549d"

QUERIES = [
    "Elon Musk politics",
    "Battery Technology",
    "Autonomous Vehicles",
    "Cybertruck",
    "BYD",
    "Full-self driving Cars",
    "Elon on X",
    "Doge",
    "TESLA earning report"
]

NUM_RESULTS = 200
RESULTS_PER_PAGE = 10

csv_filename = "data/news_headlines_2022.csv"
# If the CSV already exists, load its Titles into a set to avoid re-appending
if os.path.exists(csv_filename):
    existing_df = pd.read_csv(csv_filename, usecols=["Title"])
    seen_titles = set(existing_df["Title"].dropna().astype(str).tolist())
    write_header = False
else:
    seen_titles = set()
    write_header = True

for query in QUERIES:
    for page in range(0, NUM_RESULTS // RESULTS_PER_PAGE):
        params = {
            "q": query,
            "engine": "google_news",
            "api_key": API_KEY,
            "hl": "en",
            "gl": "us",
            "num": RESULTS_PER_PAGE,
            "start": page * RESULTS_PER_PAGE
        }

        response = requests.get("https://serpapi.com/search", params=params)
        data = response.json()

        news_results = data.get("news_results", [])
        new_rows = []
        for item in news_results:
            title = item.get("title")
            if title and title not in seen_titles:
                seen_titles.add(title)
                new_rows.append({
                    "Title": title,
                    "Source": item.get("source"),
                    "Published": item.get("date"),
                    "Link": item.get("link")
                })

        if new_rows:
            df_new = pd.DataFrame(new_rows)
            # append new uniques only
            df_new.to_csv(csv_filename, mode="a", index=False, header=write_header)
            write_header = False  # only write header once
            print(f"Appended {len(new_rows)} new headlines (total seen: {len(seen_titles)})")
        else:
            print("No new headlines on this page.")

        time.sleep(1.5)

    print(f"\n✅ Done processing query '{query}'.")

# 📥 Download the file
from google.colab import files
files.download(csv_filename)


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Collected 100 headlines so far...
Collected 200 headlines so far...
Collected 300 headlines so far...


KeyboardInterrupt: 